In [ ]:
import json
import pandas as pd
import datasets
from tqdm import tqdm

dataset_path = "/mnt/data/wangshu/hcarag/RoG-webqsp"
dataset = datasets.load_from_disk(dataset_path)
dataset = datasets.concatenate_datasets([dataset['train'], dataset['validation'], dataset['test']])


len(dataset)

4700

In [2]:
res_list = []

for i in tqdm(range(len(dataset))):
    question = dataset[i]["question"]
    answer_list = dataset[i]["answer"]
    graph = dataset[i]["graph"]
    res_dict = {"question": question, "answers": answer_list, "graph": graph}
    res_list.append(res_dict)
    
    
qa_df = pd.DataFrame(res_list)
qa_df

100%|██████████| 4700/4700 [05:10<00:00, 15.14it/s]


,question,answers,graph
0,what is the name of justin bieber brother,[Jaxon Bieber],"[[P!nk, freebase.valuenotation.is_reviewed, Ge..."
1,what character did natalie portman play in sta...,[Padmé Amidala],"[[m.0t4tf9z, film.film_crew_gig.film, Hesher],..."
2,what country is the grand bahama island in,[Bahamas],"[[Acklins, location.administrative_division.fi..."
3,what kind of money to take to bahamas,[Bahamian dollar],"[[Hurricane Betsy, common.topic.notable_types,..."
4,what character did john noble play in lord of ...,[Denethor II],"[[John Noble, film.actor.film, m.0h1ldcx], [m...."
...,...,...,...
4695,what team did david beckham play for before la...,[Manchester United F.C.],"[[m.0cq704h, common.webpage.topic, David Beckh..."
4696,who is the current leader of france 2010,[Nicolas Sarkozy],"[[Le Moniteur Universel, book.newspaper.circul..."
4697,where was the palace of knossos located,"[Crete, Greece]","[[Erythropotamos, location.location.partially_..."
4698,where is roswell area 51,"[Lincoln County, Nevada]","[[m.04krc3y, government.government_position_he..."


In [ ]:
save_qa_df = qa_df[["question", "answers"]]
def process_gb_answer(x):
        if isinstance(x, list):
            res = "|".join(map(str, x))
            return res.lower()
base_save_path = "/mnt/data/wangshu/hcarag/WebQSP/dataset"
        
save_qa_df["label"] = save_qa_df["answers"].apply(process_gb_answer)
file_path_save_qa = f"{base_save_path}/webqsp_qa.csv"
save_qa_df.to_json(file_path_save_qa, orient='records', lines=True)

save_qa_df.head(2)

,question,answers,label
0,what is the name of justin bieber brother,[Jaxon Bieber],jaxon bieber
1,what character did natalie portman play in sta...,[Padmé Amidala],padmé amidala


In [4]:
import os
base_save_path = "/mnt/data/wangshu/hcarag/WebQSP/dataset"

triples = []
for i in tqdm(range(len(qa_df))):
    graph = qa_df.iloc[i]["graph"]
    for edge in graph:
        triples.append(edge)

print(len(triples))


100%|██████████| 4700/4700 [00:01<00:00, 2573.60it/s]

19986134


In [7]:
# 使用集合去重
unique_triples = list(set(tuple(triple) for triple in triples))

# 将元组转换回列表
unique_triples = [list(triple) for triple in unique_triples]

for triple in unique_triples:
    if len(triple) != 3:
        print(triple)
        print("error")
triples[0]

['P!nk', 'freebase.valuenotation.is_reviewed', 'Gender']

In [10]:
print(len(unique_triples))

entity_set = set()
relation_set = set()
for triple in tqdm(unique_triples):
    if triple[1] not in relation_set:
        relation_set.add(triple[1])
    
    if triple[0] not in entity_set:
        entity_set.add(triple[0])
    if triple[2] not in entity_set:
        entity_set.add(triple[2])
    
entity_list = list(entity_set)
relation_list = list(relation_set)

len(entity_list), len(relation_list)

3791303


  0%|          | 0/3791303 [00:00<?, ?it/s]

100%|██████████| 3791303/3791303 [00:04<00:00, 854204.29it/s]


(1298306, 6094)

In [12]:
kg_save_path = f"{base_save_path}/triples.txt"
with open(kg_save_path, "w") as f:
    for triple in unique_triples:
        f.write("\t".join(triple) + "\n")
print("save kg done")

save kg done


In [29]:
mid_cnt = 0
res_list = []

entity_to_id = {}
for i, entity in tqdm(enumerate(entity_list), total=len(entity_list)):
    tmp = 0
    if entity.count(".") == 1:
        start_char = entity.split(".")[0]
        if len(start_char) == 1:
            mid_cnt += 1
            tmp = 1
    tmp_dict = {"entity": entity, "mid_type": tmp, "human_readable_id": i}
    res_list.append(tmp_dict)
    entity_to_id[entity] = i

entity_df = pd.DataFrame(res_list)

triple_list = []
for i, triple in tqdm(enumerate(unique_triples), total=len(unique_triples)):
    source = triple[0]
    relation = triple[1]
    target = triple[2]
    head_id = entity_to_id[source]
    tail_id = entity_to_id[target]
    tmp_dict = {
        "source": source,
        "relation": relation,
        "target": target,
        "head_id": head_id,
        "human_readable_id": i,
        "tail_id": tail_id,
    }
    triple_list.append(tmp_dict)

triple_df = pd.DataFrame(triple_list)
print(triple_df.head(2))

print(mid_cnt)
entity_df.head(2)

100%|██████████| 3791303/3791303 [00:14<00:00, 257466.16it/s]


                                        source  \
0  Soviet Union men's national volleyball team   
1                               Dorothy Dehner   

                           relation  \
0  sports.sports_team.championships   
1        common.topic.notable_types   

                                        target  head_id  human_readable_id  \
0  1987 Men's European Volleyball Championship   752166                  0   
1                                Visual Artist   320428                  1   

   tail_id  
0   281297  
1   432497  
777410


,entity,mid_type,human_readable_id
0,g.120vn5wm,1,0
1,Niko.jpg,0,1


In [30]:
out_degree_counts = triple_df['head_id'].value_counts()

in_degree_counts = triple_df['tail_id'].value_counts()

degree_counts = out_degree_counts.add(in_degree_counts, fill_value=0)


entity_df['out_degree'] = entity_df['human_readable_id'].map(out_degree_counts).fillna(0).astype(int)
entity_df['in_degree'] = entity_df['human_readable_id'].map(in_degree_counts).fillna(0).astype(int)

# 将结果添加到 entity_df
entity_df['degree'] = entity_df['human_readable_id'].map(degree_counts).fillna(0).astype(int)

print(entity_df.head(2))
# 创建 mid 到 human_readable_id 的映射字典
id_to_degree = dict(zip(entity_df["human_readable_id"], entity_df["degree"]))

triple_df["source_degree"] = triple_df["head_id"].map(id_to_degree)
triple_df["target_degree"] = triple_df["tail_id"].map(id_to_degree)
triple_df.head(2)


       entity  mid_type  human_readable_id  out_degree  in_degree  degree
0  g.120vn5wm         1                  0           0          1       1
1    Niko.jpg         0                  1           0          1       1


,source,relation,target,head_id,human_readable_id,tail_id,source_degree,target_degree
0,Soviet Union men's national volleyball team,sports.sports_team.championships,1987 Men's European Volleyball Championship,752166,0,281297,38,1
1,Dorothy Dehner,common.topic.notable_types,Visual Artist,320428,1,432497,10,1054


In [31]:
save_entity_path = f"{base_save_path}/entity_df.csv"
save_triple_path = f"{base_save_path}/relation_df.csv"

entity_df.to_csv(save_entity_path, index=False)
triple_df.to_csv(save_triple_path, index=False)
print("done")

done


In [32]:
triple_df.shape

(3791303, 8)